### Importing Libraries

In [14]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import glob 
import os
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
import numpy as np


#### Setting up device for GPU usage

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
#lang = 'english'
#STOPWORDS = stopwords.words(LANG)

#### Setting display options

In [4]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 0)

### Import Train, Val & Test Sets

In [5]:
train_data = pd.read_csv('./data/Train.csv')
val_data = pd.read_csv('./data/Valid.csv')
test_data = pd.read_csv('./data/Test.csv')

In [6]:
train_data.label.value_counts()

0    20019
1    19981
Name: label, dtype: int64

In [7]:
val_data.label.value_counts()

1    2514
0    2486
Name: label, dtype: int64

In [8]:
test_data.label.value_counts()

1    2505
0    2495
Name: label, dtype: int64

In [9]:
train_data.sample(10)

text  \
4779   This movie is not about the soda nor is it quite the French Connection.<br /><br />The Seven Ups are a group of elite policemen that use tactics not in accordance with protocol of the NYPD. Scheider heads the group with his posse or regular looking joes. They are running surveillance on a local costra nostra cartel and things go awry when a cop's wire is found out.<br /><br />Meanwhile, Richard Lynch, the most evil looking man in film (Invasion:America, Little Nikita) and his partner end up killing the cop by accident and escape from Scheider in the coolest chase scene I've seen, Bullitt and French Connection are not as good as they one up the West Side to the George Washington and onto the Palisades Parkway in New Jersey. <br /><br />The stunt drivers are terrific and Lynch makes it away free though he looks scared witless from the dangerous trip. Roy Scheider is nearly killed when his car slams into the abutted rear of Mack truck ripping the roof of his vehicle off. <br /><br />Things come to a head and one has to keep watching to follow up on such a sequence. Quick moving and intense, fresh for a thirty years.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

###### Let's take a look at some statistical figures for word count. 

In [10]:
train_data['text']= train_data['text'].astype(str)
train_data['word_count'] = train_data['text'].apply(lambda x : len(x.split()))

In [11]:
train_data.word_count.describe()

count    40000.000000
mean     231.339250  
std      171.194123  
min      4.000000    
25%      126.000000  
50%      173.000000  
75%      282.000000  
max      2470.000000 
Name: word_count, dtype: float64

### Train SVC model 

In [12]:
TF_PARAMS = {
    #'tokenizer': tweet_tok.tokenize,
    'analyzer': 'word',
    'max_df': 0.5431858441280838,
    'use_idf': True,
    'smooth_idf': True,
    'sublinear_tf': True,
    'ngram_range': (1,4),
    'lowercase': True
   # 'stop_words': STOPWORDS

}

SVC_PARAMS  = {
    'fit_intercept': True,
    #'max_iter': 2900,
    'tol': 0.019114513977298204,
    'C': 0.7962740670237443,
    'class_weight': 'balanced',
    'penalty': 'l2',
    'multi_class': 'ovr'
    }

In [13]:

SVC_pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(**TF_PARAMS)),
                    ('clf', OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(**SVC_PARAMS)))),
                     
                ])

SVC_pipeline.fit(train_data.text, train_data.label)
y_preds = SVC_pipeline.predict(test_data.text)
report = classification_report(test_data.label, y_preds)
print(str(np.mean(test_data.label == y_preds)))
print(report)



0.9094
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2495
           1       0.90      0.92      0.91      2505

    accuracy                           0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000



### Define & Wrap predict_prob function

In [15]:
def predict_proba(inputs):

    return SVC_pipeline.predict_proba(inputs)

In [16]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [17]:
wrapped_pp(['This is not a good book', 'This is a good book', 'This book is okay', 'I do not like this book'])

(array([0, 1, 0, 0], dtype=int64),
 array([[0.99174595, 0.00825405],
        [0.01146092, 0.98853908],
        [0.99761104, 0.00238896],
        [0.90312614, 0.09687386]]))

### Import pre-defined Checklist Test Suite 

In [26]:
##Import checlist
import checklist
from checklist.test_suite import TestSuite
suite_path = 'sentiment_suite.pkl'
suite = TestSuite.from_file(suite_path)

In [27]:
suite.run(wrapped_pp, overwrite = True)

Running single positive words
Predicting 19 examples
Running single negative words
Predicting 23 examples
Running Sentiment-laden words in context
Predicting 5994 examples
Running simple negations: negative
Predicting 4374 examples
Running simple negations: not negative
Predicting 4698 examples
Running conflicting statements about past and present
Predicting 2800 examples


In [28]:
suite.summary()

Vocabulary

single positive words
Test cases:      19
Fails (rate):    0 (0.0%)


single negative words
Test cases:      23
Fails (rate):    5 (21.7%)

Example fails:
0.7 unhappy
----
0.7 creepy
----
0.9 rough
----


Sentiment-laden words in context
Test cases:      5994
Fails (rate):    1105 (18.4%)

Example fails:
0.9 This series is unpleasant.
----
0.0 This script was exciting.
----
0.0 The script is exceptional.
----




Temporal

conflicting statements about past and present
Test cases:      2800
Fails (rate):    1829 (65.3%)

Example fails:
0.8 I used to think this movie was exceptional, but now I think it is sad.
----
1.0 I think this actor is hard, although in the past I thought he was fantastic.
----
1.0 I think this actor is rough,  in the past I thought he was happy.
----




Negation

simple negations: negative
Test cases:      4374
Fails (rate):    2698 (61.7%)

Example fails:
0.6 I would never say I recommend that crew.
----
0.9 I don't think I like this series.
----
1.0 

In [29]:
suite.visual_summary_table()

Please wait as we prepare the table data...


SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0}, test_infos=[{'name': 'single positive word…